Data Preperation Steps

In [34]:
import pandas as pd

# The dataset is loaded into a pandas DataFrame for analysis.
df = pd.read_csv('rawpvr_2018-02-01_28d_1083 TueFri.csv')

# I then check for any missing values in each column of the DataFrame, ensuring awareness of the columns that require imputation.
# Any missing values will be handled/imputated appropriately as they can affect statistical calculations later.
print('Columns: Number of missing values')
display(df.isna().sum())

# Impute the missing values by filling them with the median value of each column.
# We use the median as it's less sensitive to outliers, unlike the mean, which could be skewed by extreme values.
# This ensures that the calculations for speed, headway, and gap are not affected by missing data.
df['Speed (mph)'].fillna(df['Speed (mph)'].median(), inplace=True)
df['Headway (s)'].fillna(df['Headway (s)'].median(), inplace=True)
df['Gap (s)'].fillna(df['Gap (s)'].median(), inplace=True)

# The 'Date' column contains date and time information in string format.
# To facilitate time-based filtering (i.e. selecting data for specific hours), we need to convert it to datetime format.
df['Date'] = pd.to_datetime(df['Date'])

# The values of the 'Flags' column are all 0's and are updated to represent the day of the week as an integer (2 = Tuesday, 5 = Friday).
# This makes it easier to filter data by specific days when calculating the statistics for Tuesdays and Fridays.
df['Flags'] = df['Date'].dt.dayofweek + 1 

# The 'Flag Text' column is missing all of its values and is updated by storing the day of the week as a string (i.e. Monday and Tuesday ) by referring to the Flags column.
# This update improves the readability of the DataFrame when filtering or grouping by specific days.
df['Flag Text'] = df['Date'].dt.day_name()

# After handling the missing values, I verify again to ensure there are no missing values left in the DataFrame.
print('Columns: Number of missing values')
display(df.isna().sum())

Columns: Number of missing values


Date                   0
Lane                   0
Lane Name              0
Direction              0
Direction Name         0
Speed (mph)           19
Headway (s)         9992
Gap (s)            14075
Flags                  0
Flag Text         503768
dtype: int64

Columns: Number of missing values


Date              0
Lane              0
Lane Name         0
Direction         0
Direction Name    0
Speed (mph)       0
Headway (s)       0
Gap (s)           0
Flags             0
Flag Text         0
dtype: int64

In [35]:
# This function calculates various dispersion metrics (Range, Quartiles, IQR) for a given group of speed values.
# It returns a pandas series containing the measures of data dispersion (Range, Q1, Q2, Q3 and IQR) for easier use/integration with other grouped data for analysis.
# These measures are essential for understanding the distribution of speeds across lanes in Task 2.
def calculate_dispersion(group):
    Q1 = group.quantile(0.25)
    Q2 = group.quantile(0.5)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    R = group.max() - group.min()
    return pd.Series({
        'Range (R)': R,
        '1st Quartile (Q1)': Q1,
        '2nd Quartile (Q2)': Q2,
        '3rd Quartile (Q3)': Q3,
        'Interquartile Range (IQR)': IQR
    })

# This function detects outliers in a DataFrame column using the Interquartile Range (IQR) method.
# The IQR method is chosen because it is robust and not sensitive to extreme values, making it ideal for detecting outliers in skewed distributions.
# The outliers identified will be handled in another function to ensure they do not distort the dispersion metrics and skew the analysis.
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.25 * IQR
    upper_bound = Q3 + 1.25 * IQR

    # The lower and upper bounds are defined for outlier detection using a multiplier of 1.25.
    # A 1.25 multiplier helps identify outliers without being too strict (removing too much data) or too lenient (keeping extreme values).
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    # The return is a DataFrame containing only the rows that have been idenitifed as outliers.
    return outliers

# This function removes outliers (i.e. values of speed that fall outside the typical data range) from the given speed column by using the IQR method.
# Task 2 requires accurate dispersion measures when calculating the speed statistics , which is why removing outliers is an essential data preparation step that prevents skewed results when calculating speed statistics.
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.25 * IQR
    upper_bound = Q3 + 1.25 * IQR

    # The DataFrame is then filtered to remove rows where the speed value is an outlier.
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    # The cleaned DataFrame will be used to calculate more accurate/reliable speed statistics for each lane.
    return df_cleaned

Task 2 (i)

In [36]:

# The data frame is filtered for Tuesday mornings using the 'Flags' column, which represents the day of the week as an integer (2 = Tuesday).
# Using the 'Flags' column (which stores days as integers) is faster and more memory-efficient for filtering with the 'Flags Text' column (which stores days as Strings).
df_tuesday_morning = df[(df['Flags'] == 2) & 
                        (df['Date'].dt.time >= pd.to_datetime('09:00:00').time()) & 
                        (df['Date'].dt.time < pd.to_datetime('10:00:00').time())]

# The dataframe is then filtered to select only the data for the North lanes.
# The 'Direction' column, which stores integers (1 for North, 2 for South) is used for filtering instead of the 'Direction Name' which stores the days as String.
# Similar to the previous dataframe, filtering operations through integers are more efficient than strings.
df_north_lanes = df_tuesday_morning[df_tuesday_morning['Direction'] == 1]

# The outliers are detected and displayed in the original data for reference.
# This step allows me to see which values were considered outliers and removed before calculating the 'North' related statistics.
display(detect_outliers_iqr(df_north_lanes, 'Speed (mph)'))

# The outliers are removed from the 'Speed (mph)' column using the IQR method to ensure that the dispersion statistics are not distorted by extreme speed values.
# This ensures that the statistics calculated for each lane represent typical traffic conditions.
df_north_cleaned = remove_outliers_iqr(df_north_lanes, 'Speed (mph)')

# The dispersion statistics are calculated for each North lane after outliers have been removed, generating the descriptive statistics for the speed of vehicles in the North lanes on Tuesday mornings.
north_result = df_north_cleaned.groupby('Lane')['Speed (mph)'].apply(calculate_dispersion)

display(north_result)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text
78915,2018-02-06 09:01:15.000,3,NB_OS,1,North,42.875,43.800,43.457,2,Tuesday
78919,2018-02-06 09:01:18.090,1,NB_NS,1,North,42.875,5.900,5.623,2,Tuesday
79126,2018-02-06 09:03:39.030,2,NB_MID,1,North,13.048,4.286,2.253,2,Tuesday
79153,2018-02-06 09:03:50.090,2,NB_MID,1,North,13.670,4.255,2.640,2,Tuesday
79158,2018-02-06 09:03:52.080,2,NB_MID,1,North,12.428,2.610,1.229,2,Tuesday
...,...,...,...,...,...,...,...,...,...,...
462030,2018-02-27 09:57:25.020,3,NB_OS,1,North,50.331,20.100,19.762,2,Tuesday
462031,2018-02-27 09:57:25.060,2,NB_MID,1,North,43.495,2.597,2.383,2,Tuesday
462162,2018-02-27 09:59:48.000,3,NB_OS,1,North,50.331,17.500,17.213,2,Tuesday
462163,2018-02-27 09:59:49.000,2,NB_MID,1,North,45.982,8.000,7.688,2,Tuesday


Lane                           
1     Range (R)                    27.961
      1st Quartile (Q1)            24.233
      2nd Quartile (Q2)            28.584
      3rd Quartile (Q3)            31.691
      Interquartile Range (IQR)     7.458
2     Range (R)                    27.961
      1st Quartile (Q1)            25.476
      2nd Quartile (Q2)            29.205
      3rd Quartile (Q3)            32.310
      Interquartile Range (IQR)     6.834
3     Range (R)                    27.961
      1st Quartile (Q1)            27.340
      2nd Quartile (Q2)            31.069
      3rd Quartile (Q3)            34.176
      Interquartile Range (IQR)     6.836
Name: Speed (mph), dtype: float64

Task 2 (ii)

In [45]:
# Similar to the previous task, the dataframe is instead filtered to select data for the South lanes.
# As in the previous step, the 'Direction' column (integers: 1 for North, 2 for South) is used for filtering, as integers are more efficient for filtering operations than strings ('Direction Name').
df_south_lanes = df_tuesday_morning[df_tuesday_morning['Direction'] == 2]

# The outliers are detected and displayed in the original data for reference, allowing identification of outliers and removed before calculating the 'South' related statistics.
display(detect_outliers_iqr(df_south_lanes, 'Speed (mph)'))

# Outliers are removed from the 'Speed (mph)' column using the IQR method, just as with the North lanes, to ensure accurate/reliable statistics for typical traffic conditions.
df_south_cleaned = remove_outliers_iqr(df_south_lanes, 'Speed (mph)')

# The dispersion statistics are calculated for each South lane after outliers have been removed, generating the descriptive statistics for the speed of vehicles in the South lanes on Tuesday mornings.
south_result = df_south_cleaned.groupby('Lane')['Speed (mph)'].apply(calculate_dispersion)

display(south_result)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text
78943,2018-02-06 09:01:35.020,5,SB_MID,2,South,52.194,12.100,11.824,2,Tuesday
79046,2018-02-06 09:02:30.090,4,SB_OS,2,South,47.846,6.900,6.607,2,Tuesday
79093,2018-02-06 09:03:19.020,5,SB_MID,2,South,47.224,2.629,3.404,2,Tuesday
79171,2018-02-06 09:03:59.050,6,SB_NS,2,South,22.991,1.168,0.394,2,Tuesday
79180,2018-02-06 09:04:08.070,5,SB_MID,2,South,22.991,6.811,5.822,2,Tuesday
...,...,...,...,...,...,...,...,...,...,...
461978,2018-02-27 09:56:33.050,4,SB_OS,2,South,45.982,30.400,30.194,2,Tuesday
462017,2018-02-27 09:56:59.050,5,SB_MID,2,South,50.331,2.667,3.232,2,Tuesday
462042,2018-02-27 09:57:44.090,6,SB_NS,2,South,49.709,20.200,19.913,2,Tuesday
462053,2018-02-27 09:57:52.020,4,SB_OS,2,South,49.709,2.475,2.425,2,Tuesday


Lane                           
4     Range (R)                    20.506
      1st Quartile (Q1)            32.932
      2nd Quartile (Q2)            35.417
      3rd Quartile (Q3)            38.525
      Interquartile Range (IQR)     5.593
5     Range (R)                    21.128
      1st Quartile (Q1)            31.691
      2nd Quartile (Q2)            34.176
      3rd Quartile (Q3)            37.903
      Interquartile Range (IQR)     6.212
6     Range (R)                    20.506
      1st Quartile (Q1)            30.447
      2nd Quartile (Q2)            33.554
      3rd Quartile (Q3)            36.661
      Interquartile Range (IQR)     6.214
Name: Speed (mph), dtype: float64

Task 2 (iii)

In [56]:
# The traffic volume for each North lane is calculated by intially filtering the Dataframe for Tuesday mornings, then grouping the data by 'Lane' and counting the number of observations (where each observation represents a vehicle).
total_north_traffic_volume = df_tuesday_morning[df_tuesday_morning['Direction'] == 1].groupby('Lane').size()

# Similarly, the traffic volume is calculated for each South lane.
total_south_traffic_volume = df_tuesday_morning[df_tuesday_morning['Direction'] == 2].groupby('Lane').size()

# To calculate the traffic volume for each lane in the North direction within the given time period of all Tuesdays,the dataframe is filtered for North lanes (Direction == 1), then grouped by 'Lane' and 'Date'.
# The traffic volume is then counted through .size() then inserted into the created dataframe column 'Traffic Volume'.
grouped_north = df_tuesday_morning[df_tuesday_morning['Direction'] == 1].groupby(['Lane', df_tuesday_morning['Date'].dt.date])
north_traffic_volume = grouped_north.size().reset_index(name='Traffic Volume')

# The dispersion metrics for the North lane is then calculated by grouping by 'Lane' and'Traffic Volume'.
north_dispersion_metrics = north_traffic_volume.groupby('Lane')['Traffic Volume'].apply(calculate_dispersion)

# Calculating dispersion metrics for traffic volume for each lane in North direction.
print("Total Traffic Volume for North Lanes:")
print(total_north_traffic_volume)
print("Dispersion Metrics for Traffic Volume in North Lanes:")
print(north_dispersion_metrics)

# The similar process is then repeated with respective to the South lane.
grouped_south = df_tuesday_morning[df_tuesday_morning['Direction'] == 2].groupby(['Lane', df_tuesday_morning['Date'].dt.date])
south_traffic_volume = grouped_south.size().reset_index(name='Traffic Volume')

# Calculating dispersion metrics for traffic volume for each lane in South direction.
south_dispersion_metrics = south_traffic_volume.groupby('Lane')['Traffic Volume'].apply(calculate_dispersion)

# Display combined results for South lanes.
print("\nTotal Traffic Volume for South Lanes:")
print(total_south_traffic_volume)
print("\nDispersion Metrics for Traffic Volume in South Lanes:")
print(south_dispersion_metrics)

Total Traffic Volume for North Lanes:
Lane
1    2913
2    3389
3    3417
dtype: int64
Dispersion Metrics for Traffic Volume in North Lanes:
Lane                           
1     Range (R)                     97.00
      1st Quartile (Q1)            703.00
      2nd Quartile (Q2)            726.00
      3rd Quartile (Q3)            751.25
      Interquartile Range (IQR)     48.25
2     Range (R)                     56.00
      1st Quartile (Q1)            829.00
      2nd Quartile (Q2)            843.50
      3rd Quartile (Q3)            861.75
      Interquartile Range (IQR)     32.75
3     Range (R)                    109.00
      1st Quartile (Q1)            812.75
      2nd Quartile (Q2)            848.00
      3rd Quartile (Q3)            889.50
      Interquartile Range (IQR)     76.75
Name: Traffic Volume, dtype: float64

Total Traffic Volume for South Lanes:
Lane
4    1665
5    1985
6    2541
dtype: int64

Dispersion Metrics for Traffic Volume in South Lanes:
Lane               

Task 3 (i)

In [57]:
# The dataframe is filtered to include only the records for Tuesday (Flags == 2) and Friday (Flags == 5) and only between the times of 07:00:00 and 23:59:59, excluding early morning hours.
df_tue_fri = df[(df['Flags'].isin([2, 5])) & 
                (df['Date'].dt.time >= pd.to_datetime('07:00:00').time()) & 
                (df['Date'].dt.time <= pd.to_datetime('23:59:59').time())]

# The previous dataframe is further filtered to include Tuesday records, then grouped by each date and hour of Tuesday's
# The .size() then counts the number of records for each hour to get the hourly traffic volume.
tuesday_data = df_tue_fri[df_tue_fri['Flags'] == 2]
tuesday_hourly_counts = tuesday_data.groupby([tuesday_data['Date'].dt.date, tuesday_data['Date'].dt.hour]).size()

# `level=1` refers to grouping by the hour of the day to ignore the date component so that the average traffic volume per hour can be calcualted with .mean()
tuesday_avg_volume_per_hour = tuesday_hourly_counts.groupby(level=1).mean()

# The same process is applied for Friday, The previous dataframe is further filtered to include Friday records, then grouped by each date and hour of Friday's
# The .size() then counts the number of records for each hour to get the hourly traffic volume.
friday_data = df_tue_fri[df_tue_fri['Flags'] == 5]
friday_hourly_counts = friday_data.groupby([friday_data['Date'].dt.date, friday_data['Date'].dt.hour]).size()

# `level=1` refers to grouping by the hour of the day to ignore the date component so that the average traffic volume per hour can be calcualted with .mean()
friday_avg_volume_per_hour = friday_hourly_counts.groupby(level=1).mean()

print("The Average Traffic Volume per Hour on Tuesday:")
display(tuesday_avg_volume_per_hour)

print("\nThe Average Traffic Volume per Hour on Friday:")
display(friday_avg_volume_per_hour)

The Average Traffic Volume per Hour on Tuesday:


Date
7     4966.50
8     4862.00
9     3978.50
10    3178.75
11    3177.75
12    3408.00
13    3457.00
14    3634.00
15    4286.75
16    5430.25
17    5449.75
18    4452.00
19    2910.00
20    1963.00
21    1495.75
22    1066.50
23     587.50
dtype: float64


The Average Traffic Volume per Hour on Friday:


Date
7     4806.25
8     4773.00
9     3896.75
10    3284.50
11    3517.75
12    3869.25
13    3949.50
14    4206.75
15    4669.00
16    5362.00
17    5149.00
18    3853.00
19    2770.00
20    2024.25
21    1522.00
22    1398.00
23    1041.25
dtype: float64